In [1]:
from src.features import preprocessing
import pandas as pd
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', 20)  # or 1000
pd.set_option('display.max_colwidth', 500)  # or 199

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
huawei_config = preprocessing.HuaweiPreprocessorConfig()

usage: ipykernel_launcher.py [-h]
                             [--sequenceconfig_test_percentage SEQUENCECONFIG_TEST_PERCENTAGE]
                             [--sequenceconfig_random_test_split | --no-sequenceconfig_random_test_split]
                             [--sequenceconfig_random_state SEQUENCECONFIG_RANDOM_STATE]
                             [--sequenceconfig_flatten_x | --no-sequenceconfig_flatten_x]
                             [--sequenceconfig_flatten_y | --no-sequenceconfig_flatten_y]
                             [--sequenceconfig_max_window_size SEQUENCECONFIG_MAX_WINDOW_SIZE]
                             [--sequenceconfig_min_window_size SEQUENCECONFIG_MIN_WINDOW_SIZE]
                             [--sequenceconfig_window_overlap | --no-sequenceconfig_window_overlap]
                             [--sequenceconfig_allow_subwindows | --no-sequenceconfig_allow_subwindows]
                             [--sequenceconfig_valid_y_features [SEQUENCECONFIG_VALID_Y_FEATURES ...]]

SystemExit: 2

/home/vincenzo/anaconda3/envs/lena/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3406: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
deep_drain = preprocessing.HuaweiPreprocessorConfig()
deep_drain.drain_url_depth = 50 # default 10
deep_drain.fine_drain_log_depth: int = 50 # default 10
deep_drain.fine_drain_log_st: float = 0.80
deep_drain.coarse_drain_log_depth: int = 15 # default 4
deep_drain.coarse_drain_log_st: float = 0.5

In [5]:
coarse_drain = preprocessing.HuaweiPreprocessorConfig()
coarse_drain.drain_url_depth = 4 # default 10
coarse_drain.fine_drain_log_depth: int = 4 # default 10
coarse_drain.fine_drain_log_st: float = 0.5
coarse_drain.coarse_drain_log_depth: int = 2 # default 4
coarse_drain.coarse_drain_log_st: float = 0.1

In [6]:
sequence_preprocessor = preprocessing.ConcurrentAggregatedLogsPreprocessor(huawei_config)
deep_pp = preprocessing.ConcurrentAggregatedLogsPreprocessor(deep_drain)
coarse_pp = preprocessing.ConcurrentAggregatedLogsPreprocessor(coarse_drain)

In [14]:
full_df = sequence_preprocessor._load_log_only_data()
full_df[['Payload','fine_log_cluster_template', 'coarse_log_cluster_template']].to_csv("full.csv")
full_coarse = coarse_pp._load_log_only_data()
full_coarse[['Payload','fine_log_cluster_template', 'coarse_log_cluster_template']].to_csv("full_coarse.csv")
full_fine = deep_pp._load_log_only_data()
full_fine[['Payload','fine_log_cluster_template', 'coarse_log_cluster_template']].to_csv("full_fine.csv")

Generating DRAIN clusters from log_df: 100%|██████████| 24/24 [00:00<00:00, 20430.95it/s]


In [7]:
test = sequence_preprocessor.load_full_data()
test.to_csv("test.csv")

In [27]:
sequence_df = sequence_preprocessor.load_data()

Generating DRAIN clusters from log_df: 100%|██████████| 24/24 [00:00<00:00, 25725.35it/s]


In [28]:
deep_df = deep_pp.load_data()

Generating DRAIN clusters from log_df: 100%|██████████| 24/24 [00:00<00:00, 24648.21it/s]


In [29]:
coarse_df = coarse_pp.load_data()

Generating DRAIN clusters from log_df: 100%|██████████| 24/24 [00:00<00:00, 15277.48it/s]


In [30]:
sequence_df.to_csv("sequence.csv")
deep_df.to_csv("deep.csv")
coarse_df.to_csv("coarse.csv")

In [10]:
sequence_df.explode(list("attributes", "coarse_log_cluster_template"))

TypeError: list expected at most 1 argument, got 2

In [31]:
coarse_sequence_df = sequence_df.explode("coarse_log_cluster_template")
fine_sequence_df = sequence_df.explode("fine_log_cluster_template")
coarse_deep_df = deep_df.explode("coarse_log_cluster_template")
fine_deep_df = deep_df.explode("fine_log_cluster_template")
coarse_coarse_df = coarse_df.explode("coarse_log_cluster_template")
fine_coarse_df = coarse_df.explode("fine_log_cluster_template")

In [32]:
coarse_coarse_df["coarse_log_cluster_template"].to_csv("coarse_coarse.csv")
coarse_deep_df["coarse_log_cluster_template"].to_csv("coarse_deep.csv")
coarse_sequence_df["coarse_log_cluster_template"].to_csv("coarse_sequence.csv")
fine_deep_df["fine_log_cluster_template"].to_csv("fine_deep.csv")
fine_coarse_df["fine_log_cluster_template"].to_csv("fine_coarse.csv")
fine_sequence_df["fine_log_cluster_template"].to_csv("fine_sequence.csv")

In [2]:
pkl = pd.read_pickle("data/huawei.pkl")
pkl.head()

,Hostname,log_level,programname,python_module,http_status,http_method,@timestamp,Payload,http_url,log_cluster_template,log_cluster_path,timestamp,parent_trace_id,url_cluster_template,url_cluster_path
0,wally113,INFO,glance-api,eventlet.wsgi.server,NaN,NaN,2019-11-25T17:16:01.590000000+01:00,"172.17.0.2 - - [25/Nov/2019 17:16:01] ""GET /v2/schemas/image HTTP/1.1"" 200 5562 0.053853",NaN,NaN,NaN,2019-11-25 17:16:01.590000+01:00,000444b6-b3bc-40aa-8a89-ee2486186b83,NaN,NaN
1,wally113,INFO,glance-api,eventlet.wsgi.server,NaN,NaN,2019-11-25T17:16:01.724000000+01:00,"172.17.0.2 - - [25/Nov/2019 17:16:01] ""POST /v2/images HTTP/1.1"" 201 1002 0.116297",NaN,NaN,NaN,2019-11-25 17:16:01.724000+01:00,000444b6-b3bc-40aa-8a89-ee2486186b83,NaN,NaN
2,wally113,INFO,glance-api,eventlet.wsgi.server,NaN,NaN,2019-11-25T17:16:07.687000000+01:00,"172.17.0.2 - - [25/Nov/2019 17:16:07] ""GET /v2/images/e67a3a0b-4186-4996-8cd2-b730298af506 HTTP/1.1"" 200 1133 0.043425",NaN,25 nov 2019 17 16 * get v2 images * http 11 200 1133 *,15 * nov * * * * get v2 images,2019-11-25 17:16:07.687000+01:00,000444b6-b3bc-40aa-8a89-ee2486186b83,NaN,NaN
3,wally113,INFO,neutron-openvswitch-agent,neutron.plugins.ml2.drivers.openvswitch.agent.ovs_neutron_agent,NaN,NaN,2019-11-25T17:16:08.075000000+01:00,"SIGTERM received, capping RPC timeout by 10 seconds.",NaN,sigterm received capping rpc timeout by 10 seconds,8 sigterm received capping rpc timeout by * seconds,2019-11-25 17:16:08.075000+01:00,000444b6-b3bc-40aa-8a89-ee2486186b83,NaN,NaN
4,wally113,INFO,glance-api,eventlet.wsgi.server,NaN,NaN,2019-11-25T17:16:26.982000000+01:00,"172.17.0.2 - - [25/Nov/2019 17:16:26] ""GET /v2/schemas/image HTTP/1.1"" 200 5562 0.006188",NaN,NaN,NaN,2019-11-25 17:16:26.982000+01:00,000444b6-b3bc-40aa-8a89-ee2486186b83,NaN,NaN


In [2]:
df = pd.read_pickle("../data/sequences_df.pkl")

In [3]:
df

,num_logs,num_events,all_events,attributes,fine_log_cluster_template,coarse_log_cluster_template
0,182,182,"[[fine_log_cluster_template#25 nov 2019 15 * * 0100 get resource providers * * http 11 200 * * novacompute keystoneauth1 3131 pythonrequests 2210 cpython 2715, programname#placement-api-access, , url_cluster_template#resource providers 45bac5db-7b40-4922-ae54-fe747131258d allocations, http_method#get, , Hostname#wally113, coarse_log_cluster_template#25 nov 2019 15 * * 0100 get resource providers * * http 11 200 * * novacompute keystoneauth1 3131 pythonrequests 2210 cpython 2715, http_status#...","[[programname#placement-api-access, , url_cluster_template#resource providers 45bac5db-7b40-4922-ae54-fe747131258d allocations, http_method#get, , Hostname#wally113, http_status#200.0], [programname#keystone-apache-public-access, , , http_method#get, , Hostname#wally113, http_status#300.0], [programname#keystone-apache-public-access, , url_cluster_template#v3 auth tokens, http_method#post, , Hostname#wally113, http_status#201.0], [programname#keystone-apache-public-access, , url_cluster_temp...","[[fine_log_cluster_template#25 nov 2019 15 * * 0100 get resource providers * * http 11 200 * * novacompute keystoneauth1 3131 pythonrequests 2210 cpython 2715], [fine_log_cluster_template#25 nov 2019 15 * * 0100 get http 11 300 270 * rally keystoneauth1 3110 pythonrequests 2210 cpython 2712], [fine_log_cluster_template#25 nov 2019 15 * * 0100 post v3 auth tokens http 11 201 * * rally keystoneauth1 3110 pythonrequests 2210 cpython 2712], [fine_log_cluster_template#25 nov 2019 15 * * 0100 post...","[[coarse_log_cluster_template#25 nov 2019 15 * * 0100 get resource providers * * http 11 200 * * novacompute keystoneauth1 3131 pythonrequests 2210 cpython 2715], [coarse_log_cluster_template#25 nov 2019 15 * * 0100 * * * * * * * * * * * * *], [coarse_log_cluster_template#25 nov 2019 15 * * 0100 post v3 auth tokens http 11 201 * * rally keystoneauth1 3110 pythonrequests 2210 cpython 2712], [coarse_log_cluster_template#25 nov 2019 15 * * 0100 post v3 auth tokens http 11 201 * * rally keystone..."
